# Segmenting and Clustering Neighborhoods in the city of Toronto, Canada

Let's download all the dependencies that we will need.

In [ ]:
import pandas as pd
import numpy as np
import random
import requests

from bs4 import BeautifulSoup

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#!pip install geopy
from geopy.geocoders import Nominatim
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize

#!pip install folium==0.5.0
import folium



# Phase 1: Creating dataframe with postal codes

Using Beautiful Soup to get postal code

In [ ]:
Code_Canada = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').content

In [ ]:
soup = BeautifulSoup(Code_Canada,'html.parser')

Create a DataFrame from the necessary content on the page

In [ ]:
table_code =[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_code.append(cell)

#### Tranform the data into a _pandas_ dataframe

In [ ]:
df_code =pd.DataFrame(table_code)
df_code

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [ ]:
df_code['Borough']

0                                             North York
1                                             North York
2                                       Downtown Toronto
3                                             North York
4                                           Queen's Park
                             ...                        
98                                             Etobicoke
99                                      Downtown Toronto
100    East TorontoBusiness reply mail Processing Cen...
101                                            Etobicoke
102                                            Etobicoke
Name: Borough, Length: 103, dtype: object

#### Renaming instances in the column Borough

In [ ]:

df_code['Borough']=df_code['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


In [ ]:
df_code['Borough']

0                 North York
1                 North York
2           Downtown Toronto
3                 North York
4               Queen's Park
               ...          
98                 Etobicoke
99          Downtown Toronto
100    East Toronto Business
101                Etobicoke
102                Etobicoke
Name: Borough, Length: 103, dtype: object

In [ ]:
df_code.shape

(103, 3)

# Phase 2: Getting latitude and longitude coordinates

#### Obtaining geocode through csv file

In [ ]:
geocode_csv = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
geocode_csv.rename(columns={"Postal Code": "PostalCode"}, inplace=True)


#### Merge 
#### Quickly examine the resulting dataframe

In [ ]:
df_geocode = pd.merge(df_code, geocode_csv, how ='right', on ='PostalCode')
df_geocode

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


# Phase 3: Exploring and grouping Toronto neighborhoods


#### Neighborhoods that contain the word Toronto

In [ ]:
toronto_data = df_geocode[df_geocode['Borough'].str.contains('Toronto')]
toronto_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
40,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160


In [ ]:
toronto_data['Borough'].value_counts()

Downtown Toronto          17
Central Toronto            9
West Toronto               6
East Toronto               4
Downtown Toronto Stn A     1
East York/East Toronto     1
East Toronto Business      1
Name: Borough, dtype: int64

Toronto location on the map using Folium



In [ ]:
lat = 43.651070
lon = -79.347015

Map_Toronto = folium.Map(location=[lat, lon],zoom_start=11)

folium.CircleMarker(
                    [lat, lon],
                    radius=5,
                    popup='Toronto',
                    fill=True,
                    color='red',
                    fill_color='red',
                    fill_opacity=0.7).add_to(Map_Toronto)
Map_Toronto

#### Let's visualizate Toronto the neighborhoods in it.


In [ ]:


for lat, lon, borough, neighborhood in zip(toronto_data['Latitude'],toronto_data['Longitude'],toronto_data['Borough'],toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.5
        ).add_to(Map_Toronto)
Map_Toronto

#### Run k-means to cluster the neighborhood into 5 clusters

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
k=5
toronto_data2 = toronto_data
borough_clusters = toronto_data2.drop(['PostalCode','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(borough_clusters)
kmeans.labels_
toronto_data2.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_data2.head()

,Cluster Labels,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,4,M4E,East Toronto,The Beaches,43.676357,-79.293031
40,4,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106
41,4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,4,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,4,M4M,East Toronto,Studio District,43.659526,-79.340923


#### Finally, let's visualize the resulting clusters

In [ ]:
# create map
lat = 43.651070
lon = -79.347015
kclusters = 5
map_clusters = folium.Map(location=[lat, lon], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighborhood, cluster in zip(toronto_data2['Latitude'],toronto_data2['Longitude'],toronto_data2['Neighborhood'],toronto_data2['Cluster Labels']):
    label = '{}, {}, Cluster {}'.format(neighborhood, borough, cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

       
map_clusters